# Importing library and Loading Datasets

In [ ]:
import pandas as pd

In [3]:
import os

In [4]:
os.getcwd()

'C:\\Users\\USER VICTUS'

In [5]:
os.chdir("C:\\Users\\USER VICTUS\\Downloads")

In [6]:
orders=pd.read_csv("orders.csv")

In [7]:
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [8]:
users=pd.read_json("users.json")

In [9]:
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [10]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)



In [11]:
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [12]:
orders.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name'],
      dtype='object')

In [13]:
users.columns

Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [14]:
restaurants.columns

Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

# Merging Datasets

In [15]:
df = pd.merge(orders, users, on="user_id", how="left")
df = pd.merge(df, restaurants, on="restaurant_id", how="left")


In [16]:
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [17]:
df.shape

(10000, 12)

In [18]:
df.to_csv("final_food_delivery_dataset.csv", index=False)

## MCQ-Based Analysis & Insights
The following section contains data analysis performed to answer the given MCQ questions.

In [19]:
df_gold = df[df['membership'] == 'Gold']

df_gold.groupby('city')['total_amount'].sum().sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [20]:
df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [21]:
user_spend = df.groupby('user_id')['total_amount'].sum()

user_spend[user_spend > 1000].count()


np.int64(2544)

In [22]:
rating_bins = pd.cut(
    df['rating'],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0]
)

df.groupby(rating_bins)['total_amount'].sum()


C:\Users\USER VICTUS\AppData\Local\Temp\ipykernel_20060\735996976.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(rating_bins)['total_amount'].sum()


rating
(3.0, 3.5]    1881754.57
(3.5, 4.0]    1717494.41
(4.0, 4.5]    1960326.26
(4.5, 5.0]    2197030.75
Name: total_amount, dtype: float64

In [23]:
df_gold.groupby('city')['total_amount'].mean().sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [24]:
df.groupby('cuisine')['restaurant_id'].nunique().sort_values()

cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64

In [25]:
df.groupby('cuisine')['total_amount'].sum().sort_values(ascending=False)


cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64

In [26]:
total_orders = df['order_id'].nunique()
gold_orders = df[df['membership']=='Gold']['order_id'].nunique()

percentage = (gold_orders / total_orders) * 100
round(percentage)


50

In [27]:
df['restaurant_name'] = df['restaurant_name_x'].combine_first(df['restaurant_name_y'])

rest_stats = df.groupby('restaurant_name').agg(
    avg_order_value=('total_amount', 'mean'),
    total_orders=('order_id', 'count')
)

rest_stats_filtered = rest_stats[rest_stats['total_orders'] < 20] \
    .sort_values('avg_order_value', ascending=False)

print(rest_stats_filtered)


                                   avg_order_value  total_orders
restaurant_name                                                 
Hotel Dhaba Multicuisine               1040.222308            13
Sri Mess Punjabi                       1029.180833            12
Ruchi Biryani Punjabi                  1002.140625            16
Sri Delights Pure Veg                   989.467222            18
Classic Kitchen Family Restaurant       973.167895            19
...                                            ...           ...
Annapurna Tiffins Punjabi               621.828947            19
Darbar Tiffins Non-Veg                  596.815556            18
Darbar Restaurant Punjabi               589.972857            14
Spice Tiffins Pure Veg                  578.578667            15
Ruchi Mess Punjabi                      572.686471            17

[170 rows x 2 columns]


In [28]:
# "Based on the dataset, the restaurant with the highest average order value (<20 orders) is Hotel Dhaba Multicuisine which is not in the given options.”
# So analyzing using mcq options Top restaurant based on dataset with <20 orders and highest average order value, this is the closest match.”

In [29]:
options = [
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]

top_option = rest_stats_filtered.loc[
    rest_stats_filtered.index.isin(options), 'avg_order_value'
].idxmax()

print("Choose this MCQ option:", top_option)


Choose this MCQ option: Ruchi Foods Chinese


In [30]:
df.groupby(['membership','cuisine'])['total_amount'] \
  .sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [31]:
# “Based on the dataset, Regular + Mexican combination has the highest revenue, but among the given options, Regular + Indian cuisine contributes the highest revenue.”

In [32]:
df['order_date'] = pd.to_datetime(df['order_date'])

df.groupby(df['order_date'].dt.to_period('Q'))['total_amount'] \
  .sum().sort_values(ascending=False)


C:\Users\USER VICTUS\AppData\Local\Temp\ipykernel_20060\1013929788.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


order_date
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [33]:
gold_orders_count = df[df['membership']=='Gold']['order_id'].nunique()
gold_orders_count


4987

In [34]:
hyderabad_revenue = df[df['city']=='Hyderabad']['total_amount'].sum()
round(hyderabad_revenue)


1889367

In [35]:
distinct_users = df['user_id'].nunique()
distinct_users

2883

In [36]:
avg_gold_order = df[df['membership']=='Gold']['total_amount'].mean()
round(avg_gold_order, 2)

np.float64(797.15)

In [37]:
high_rating_orders = df[df['rating'] >= 4.5]['order_id'].nunique()
high_rating_orders

3374

In [38]:
df_gold = df[df['membership']=='Gold']
top_gold_city = df_gold.groupby('city')['total_amount'].sum().idxmax()
top_gold_city

'Chennai'

In [39]:
top_city_orders = df_gold[df_gold['city'] == top_gold_city]['order_id'].nunique()
top_city_orders


1337

In [40]:
len(df)

10000

In [41]:
df.describe()

,order_id,user_id,restaurant_id,order_date,total_amount,rating
count,10000.00000,10000.000000,10000.000000,10000,10000.000000,10000.000000
mean,5000.50000,1504.117700,251.016700,2023-07-02 11:45:53.280000,801.162412,4.045430
min,1.00000,1.000000,1.000000,2023-01-01 00:00:00,100.200000,3.000000
25%,2500.75000,761.000000,127.000000,2023-04-01 00:00:00,446.310000,3.500000
50%,5000.50000,1508.000000,251.000000,2023-07-03 00:00:00,806.295000,4.100000
75%,7500.25000,2250.250000,376.000000,2023-10-02 00:00:00,1149.227500,4.600000
max,10000.00000,3000.000000,500.000000,2024-01-01 00:00:00,1499.830000,5.000000
std,2886.89568,861.727776,144.622558,NaN,405.458753,0.606531


## Final Conclusion

This analysis examined restaurant performance, cuisine trends, and customer behavior using the food delivery dataset.  
Key insights were derived through aggregation and filtering to answer analytical and MCQ-based questions.

The notebook highlights revenue patterns across cuisines and membership types, and identifies high-value restaurants with fewer orders.  
Overall, the results are data-driven, clearly justified, and supported by reproducible calculations.
